In [1]:
!pip install bert

  Created wheel for bert: filename=bert-2.2.0-cp36-none-any.whl size=3756 sha256=53480f368e8480b402370ec77498283165c606fc54792426d6f4f50f393b9a41
  Stored in directory: /root/.cache/pip/wheels/fe/71/b7/941459453bd38e5d97a8c886361dee19325e9933c9cf88ad46
  Created wheel for erlastic: filename=erlastic-2.0.0-cp36-none-any.whl size=6786 sha256=9dd2c606be5a15898cd83ba8b05d69ff881a45dc818daed617a16b8168e245bf
  Stored in directory: /root/.cache/pip/wheels/02/62/46/93c713a5f061aeeb4f16eb6bf5ee798816e6ddda70faa78e69
Successfully built bert erlastic


In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 3.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30315 sha256=b1041498e9bd3f9ca1e4751f609bae096197ebf4ef61bd6810722befc6960eb9
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=6e2f9ae4389d9a03f001f6197379db98759376115363b2a09deaf720b5be7ffe
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=0d87dfb9c65b90bd0b7da945b82c0dce1af0ba47deefc539ec37be64557d1c95
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 7.2MB/s 


In [3]:
# Import modules
import pandas as pd
import numpy as np
import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm
import matplotlib.pyplot as plt

# string processing
import re
from keras.utils.np_utils import to_categorical
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics

print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
pd.set_option('display.max_colwidth',1000)

Using TensorFlow backend.


TensorFlow Version: 2.2.0
Hub version:  0.8.0


In [4]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Start here:

In [5]:
# to navigate to the data location
import os

# get current directory 
path = os.getcwd() 

# parent directory
parent = os.path.dirname(path)

df_merge_quality = pd.read_csv('/content/drive/My Drive/w266 Final Project /data/datasets/US_patent_abstract_50000_2015_with_title_1_5y.csv')
df_merge_quality.shape

(50000, 30)

In [6]:
df = df_merge_quality[['claims_text', 'quality_rank']]
df

,claims_text,quality_rank
0,"What is claimed is: \n \n 1. A user equipment (UE) device, the UE device comprising:\n an antenna for performing wireless communications with a base station; \n a memory which stores a plurality of sets of communication scenario information for each of a plurality of UE communication scenarios, wherein each of the stored plurality of sets of communication scenario information is based on two or more of receiver type, multiple input-multiple output (MIMO) scheme, and amount of Doppler shift, wherein each set of communication scenario information is useable in generating a channel quality indicator for a respective communication scenario; \n a processor configured to:\n determine a current communication scenario of the UE device during operation of the UE, wherein determining the current communication scenario is based on the two or more of receiver type, MIMO scheme, and amount of Doppler shift; \n select a first set of communication scenario information from the plurality...",1
1,"I claim: \n \n 1. A mechanism for assembling and connecting a furniture base with a top surface to a furniture box with at least one scissor, glider or recliner mechanism affixed thereto comprising:\n a first cam piece with a top and a bottom, said bottom seated on the top surface of said base, said first cam piece configured with a first cam interlock and a first positioning element; \n a second cam piece with a top and a bottom, securely affixed in a fixed position to a lowermost section of said at least one scissor, glider or recliner mechanism, said second cam piece configured with a second cam interlock and a second positioning element; \n wherein when said first cam piece is moved toward said second cam piece in a connection direction, with said first and second positioning elements aligned to engage each other, said first cam interlock and said second cam interlock are aligned to mate and resiliently interlock one to another, with some or all of the top of the seco...",0
2,"We claim: \n \n 1. An aircraft landing gear wheel, comprising:\n a main body, comprising; \n a hub portion including a gear end and an attachment end; \n a driven gear portion comprising a first annular rail and a second annular rail, the driven gear portion being integrally and drivingly connected with the hub portion at the gear end; \n one or more rollers rotatingly connected between roller apertures of the first annular rail and the second annular rail, and \n bearings rotatingly supporting portions of each of the one or more rollers within the roller apertures. \n \n \n \n 2. The wheel of claim 1 , wherein the main body further comprises an intermediary portion integrally connected to the gear end of the hub portion and the driven gear portion. \n \n \n 3. The wheel of claim 2 , wherein the intermediary portion comprises at least one connecting bar. \n \n \n 4. The wheel of claim 3 , wherein the at least one connecting bar...",1
3,"1. A method for culturing primate pluripotent stem cells in an undifferentiated state on a matrix without the need for feeder cells or conditioned medium, the method comprising the step of:\n culturing the primate pluripotent stem cells on an extracellular matrix in a medium without feeder cells or conditioned media, the medium comprising salts, vitamins, amino acids, glucose, albumin, minerals, lipids, a transferrin or a transferrin substitute, insulin or an insulin substitute, a fibroblast growth factor, and at least one member selected from gamma-aminobutyric acid, pipecolic acid, and lithium in sufficient amounts to maintain the cells in an undifferentiated state through multiple successive culture passages. \n \n \n \n 2. The method of claim 1 wherein the matrix comprises a solubilized basement membrane preparation extracted from the Engelbreth-Holm-Swarm mouse sarcoma or a matrix that comprises human matrix proteins collagen IV and at least one member selected...",1
4,"What is claimed is: \n \n 1. A method 

In [7]:
df['claims_text'] = df['claims_text'].apply(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
df.apply(lambda x: sum(x.isnull()), axis=0)

claims_text     0
quality_rank    0
dtype: int64

In [9]:
# Functions for constructing BERT Embeddings: input_ids, input_masks, input_segments and Inputs
MAX_SEQ_LEN=512 # max sequence length

def get_masks(tokens):
    """Masks: 1 for real tokens and 0 for paddings"""
    return [1]*len(tokens) + [0] * (MAX_SEQ_LEN - len(tokens))
 
def get_segments(tokens):
    """Segments: 0 for the first sequence, 1 for the second"""  
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (MAX_SEQ_LEN - len(tokens))

def get_ids(tokens, tokenizer):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (MAX_SEQ_LEN - len(token_ids))
    return input_ids

def create_single_input(sentence, tokenizer, max_len):
    """Create an input from a sentence"""
    stokens = tokenizer.tokenize(sentence)
    stokens = stokens[-max_len:] # last n words
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
    ids = get_ids(stokens, tokenizer)
    masks = get_masks(stokens)
    segments = get_segments(stokens)

    return ids, masks, segments
 
def convert_sentences_to_features(sentences, tokenizer):
    """Convert sentences to features: input_ids, input_masks and input_segments"""
    input_ids, input_masks, input_segments = [], [], []
 
    for sentence in tqdm(sentences,position=0, leave=True):
      ids,masks,segments=create_single_input(sentence,tokenizer,MAX_SEQ_LEN-2)
      assert len(ids) == MAX_SEQ_LEN
      assert len(masks) == MAX_SEQ_LEN
      assert len(segments) == MAX_SEQ_LEN
      input_ids.append(ids)
      input_masks.append(masks)
      input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
          np.asarray(input_masks, dtype=np.int32), 
          np.asarray(input_segments, dtype=np.int32)]

def create_tonkenizer(bert_layer):
    """Instantiate Tokenizer with vocab"""
    vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case=bert_layer.resolved_object.do_lower_case.numpy() 
    tokenizer=bert.bert_tokenization.FullTokenizer(vocab_file,do_lower_case)
    return tokenizer

In [10]:
def bert_model(callable_object):
    # Load the pre-trained BERT base model
    bert_layer = hub.KerasLayer(handle=callable_object, trainable=True)  
   
    # BERT layer three inputs: ids, masks and segments
    input_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_ids")           
    input_masks = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_masks")       
    input_segments = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="segment_ids")
    
    inputs = [input_ids, input_masks, input_segments] # BERT inputs
    pooled_output, sequence_output = bert_layer(inputs) # BERT outputs
    
    # Add a hidden layer
    x = Dense(units=768, activation='relu')(pooled_output)
    x = Dropout(0.1)(x)
 
    # Add output layer
    outputs = Dense(2, activation="sigmoid")(x)

    # Construct a new model
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = bert_model("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [11]:
df['quality_rank'] = df['quality_rank'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
# Create examples for training and testing
# df = df.sample(frac=1) # Shuffle the dataset  # already shuffle in the csv file
tokenizer = create_tonkenizer(model.layers[3])
X_train = convert_sentences_to_features(df['claims_text'][:35000], tokenizer)
X_dev = convert_sentences_to_features(df['claims_text'][35000:45000], tokenizer)
X_test = convert_sentences_to_features(df['claims_text'][45000:], tokenizer)


# for binary classification, since in the loss function I use binary_crossentropy, looks like prefer this single array
y_train = to_categorical(df['quality_rank'][:35000].values)
y_dev =  to_categorical(df['quality_rank'][35000:45000].values)
y_test =  to_categorical(df['quality_rank'][45000:].values)




100%|██████████| 5000/5000 [01:39<00:00, 50.50it/s]


In [13]:
# distribution of label value in train
np.unique([np.argmax(i) for i in y_train], return_counts=True)

(array([0, 1]), array([20815, 14185]))

In [14]:
# distribution of label value in dev
np.unique([np.argmax(i) for i in y_dev], return_counts=True)

(array([0, 1]), array([5898, 4102]))

In [15]:
# distribution of label value in test
np.unique([np.argmax(i) for i in y_test], return_counts=True)

(array([0, 1]), array([2996, 2004]))

In [16]:
# setup checkpoint

checkpoint_path = "/content/drive/My Drive/w266 Final Project /ckpt_bert_fine_tune_last_n_words"

ckpt = tf.train.Checkpoint(model = model) # https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [17]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [29]:
# Train the model
BATCH_SIZE = 10
EPOCHS = 4

# Use Adam optimizer to minimize the categorical_crossentropy loss
opt = Adam(learning_rate=2e-5)
model.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Fit the data to the model
history = model.fit(X_train, y_train,
                    validation_data=(X_dev, y_dev),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose = 1,
                    callbacks=[MyCustomCallback()])


Epoch 1/4
3500/3500 [==============================] - 4714s 1s/step - loss: 0.6025 - accuracy: 0.6737 - val_loss: 0.5913 - val_accuracy: 0.6961
Epoch 2/4
3500/3500 [==============================] - 4718s 1s/step - loss: 0.5580 - accuracy: 0.7158 - val_loss: 0.6045 - val_accuracy: 0.6974
Epoch 3/4
3500/3500 [==============================] - 4714s 1s/step - loss: 0.4824 - accuracy: 0.7718 - val_loss: 0.6104 - val_accuracy: 0.6751
Epoch 4/4
3500/3500 [==============================] - 4719s 1s/step - loss: 0.3244 - accuracy: 0.8648 - val_loss: 0.8260 - val_accuracy: 0.6471


In [42]:
model.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

loss, acc = model.evaluate(X_dev, y_dev, verbose=1)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

313/313 [==============================] - 446s 1s/step - loss: 0.7527 - accuracy: 0.4102
Untrained model, accuracy: 41.02%


In [ ]:
# disconnected, re-start training from epoch = 3

BATCH_SIZE = 10
EPOCHS = 2


# Use Adam optimizer to minimize the categorical_crossentropy loss
opt = Adam(learning_rate=2e-5)
model.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Fit the data to the model
history = model.fit(X_train, y_train,
                    validation_data=(X_dev, y_dev),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose = 1,
                    callbacks=[MyCustomCallback()])


Epoch 1/2
400/400 [==============================] - 5427s 14s/step - loss: 0.5214 - accuracy: 0.7477 - val_loss: 0.6674 - val_accuracy: 0.6540
Epoch 2/2
400/400 [==============================] - 5329s 13s/step - loss: 0.3776 - accuracy: 0.8395 - val_loss: 0.7390 - val_accuracy: 0.6510


In [18]:
ckpt_manager.latest_checkpoint

'/content/drive/My Drive/w266 Final Project /ckpt_bert_fine_tune_last_n_words/ckpt-4'

Restore the checkpoint to epoch 2

In [19]:
ckpt.restore("/content/drive/My Drive/w266 Final Project /ckpt_bert_fine_tune_last_n_words/ckpt-2")

In [20]:
pred_dev = model.predict(X_dev)
pred_dev

array([[0.6208703 , 0.38418096],
       [0.7817111 , 0.21514839],
       [0.70922405, 0.28511858],
       ...,
       [0.9578299 , 0.04130707],
       [0.7950583 , 0.20636682],
       [0.2236444 , 0.77476877]], dtype=float32)

In [21]:
np.savetxt('/content/drive/My Drive/w266 Final Project /Predict_Output/BERT_fine_tune_5yr_claims_dev_prob_last_n_words_50k_2.csv', pred_dev)

In [22]:
predicted = [np.argmax(pred) for pred in 
             pred_dev]

In [23]:
y_dev_binary = df['quality_rank'][35000:45000].values
y_dev_binary

array([0, 0, 1, ..., 0, 0, 1])

In [24]:
## Accuracy, Precision, Recall
accuracy = metrics.accuracy_score(y_dev_binary, predicted)
auc = metrics.roc_auc_score(y_dev_binary, predicted)  # predicted_prob), check doc, seems the second argument required to be shape (n_samples,) for binary case 
                            #multi_class="ovr") # check documentation and seems "ovr" not good for only binary target class
print("Accuracy:",  round(accuracy,3))
print("Auc:", round(auc,3))
print("Detail:")
print(metrics.classification_report(y_dev_binary, predicted))

# Accuracy output 0.627 => align with model performance result!

Accuracy: 0.697
Auc: 0.658
Detail:
              precision    recall  f1-score   support

           0       0.69      0.88      0.77      5898
           1       0.71      0.44      0.54      4102

    accuracy                           0.70     10000
   macro avg       0.70      0.66      0.66     10000
weighted avg       0.70      0.70      0.68     10000



In [ ]:
# let's restore the 2nd epoch

checkpoint_path = "ckpt_bert_fine_tune_last_n_words/"

ckpt = tf.train.Checkpoint(model = model) # https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=2)

if ckpt_manager.latest_checkpoint:
    ckpt.restore('ckpt_bert_fine_tune_last_n_words/ckpt-2')
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [20]:
pred_test = model.predict(X_test)
pred_test

array([[0.51496196, 0.4888194 ],
       [0.9344356 , 0.06480081],
       [0.96182865, 0.03628227],
       ...,
       [0.6571694 , 0.34459844],
       [0.95817155, 0.04035736],
       [0.91324943, 0.08616073]], dtype=float32)

In [23]:
np.savetxt('/content/drive/My Drive/w266 Final Project /Predict_Output/BERT_fine_tune_5yr_claims_test_prob_last_n_words_50k.csv', pred_test)

In [24]:
predicted_test = [np.argmax(pred) for pred in 
             pred_test]

In [25]:
y_test_binary = df['quality_rank'][45000:].values
#y_test_binary

In [27]:
## Accuracy, Precision, Recall
accuracy = metrics.accuracy_score(y_test_binary, predicted_test)
auc = metrics.roc_auc_score(y_test_binary, predicted_test)  # predicted_prob), check doc, seems the second argument required to be shape (n_samples,) for binary case 
                            #multi_class="ovr") # check documentation and seems "ovr" not good for only binary target class
print("Accuracy:",  round(accuracy,3))
print("Auc:", round(auc,3))
print("Detail:")
print(metrics.classification_report(y_test_binary, predicted_test))

# Accuracy output 0.627 => align with model performance result!

Accuracy: 0.698
Auc: 0.654
Detail:
              precision    recall  f1-score   support

           0       0.70      0.88      0.78      2996
           1       0.70      0.43      0.53      2004

    accuracy                           0.70      5000
   macro avg       0.70      0.65      0.66      5000
weighted avg       0.70      0.70      0.68      5000

